In [8]:
import csv
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.layers import Dense, Dropout
from keras import Sequential, metrics

from keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
#dataset is collected from Kaggle computition "Credit Card Fraud Detection"
dataPath = 'CreditCardFraudDetectionDataset\\creditcard.csv'

features = []
targets = []
with open(dataPath) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("Header:", line.strip()) # .strip() removes space before and after the string
            continue #skip the header
        fields = line.strip().split(",")
        features.append([float(v.replace('""', "")) for v in fields[:-1]])
        targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("\n\nExample Features:", features[-1])

features = np.array(features, dtype = "float32") # storing feature data in a numpy array
targets = np.array(targets, dtype = "uint8")
print('\n\nThe shape of feature is {} and target is {}'.format(features.shape, targets.shape))

Header: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"


Example Features: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]


The shape of feature is (284807, 30) and target is (284807, 1)


In [2]:
#Prepare training and validation dataset
valid_samples = int(len(features)*0.2) # 20% of the dataset are validation data
train_data = features[:-valid_samples]
train_targets = targets[:-valid_samples]
valid_data = features[-valid_samples:]
valid_targets = targets[-valid_samples:]

print("Number of Training samples: {} \nNumber of validation samples: {}"
      .format(len(train_data), len(valid_data)))

Number of Training samples: 227846 
Number of validation samples: 56961


In [3]:
#Analysing calss imbalance in the target data

counts = np.bincount(train_targets[:, 0])
print("Number of positive samples in training data: {} ({:0.2f}% of the total)"
      .format(counts[1], 100*float(counts[1])/len(train_targets)))

weight_for_0 = 1.0/counts[0]
weigth_for_1 = 1.0/counts[1]


Number of positive samples in training data: 417 (0.18% of the total)


In [4]:
#Normalizing the data using training set satistics
mean =  np.mean(train_data, axis = 0)
train_data -= mean
valid_data -= mean
std = np.std(train_data, axis = 0)
train_data /= std
valid_data /= std

In [5]:
#Build a binary classification model

model = Sequential(
        [Dense(100, activation = 'relu', input_shape = (train_data.shape[-1],)),
        Dense(128, activation = 'relu'),
        Dropout(0.25),
        Dense(100, activation = 'sigmoid'),
        ]
)
model.summary()
        

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               3100      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               12900     
Total params: 28,928
Trainable params: 28,928
Non-trainable params: 0
_________________________________________________________________


In [6]:
#Training the model with class_weight argument
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
class_weight = {0: weight_for_0, 1: weigth_for_1}

model.fit(
    train_data,
    train_targets,
    batch_size=2048,
    epochs=16,
    verbose=2,
    validation_data=(valid_data, valid_targets),
    class_weight=class_weight,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 227846 samples, validate on 56961 samples
Epoch 1/16
 - 2s - loss: 3.6208e-05 - acc: 0.1981 - val_loss: 4.0113 - val_acc: 0.5359
Epoch 2/16
 - 2s - loss: 2.8741e-05 - acc: 0.6913 - val_loss: 3.3827 - val_acc: 0.7773
Epoch 3/16
 - 2s - loss: 1.8693e-05 - acc: 0.8220 - val_loss: 2.0516 - val_acc: 0.8535
Epoch 4/16
 - 2s - loss: 1.0066e-05 - acc: 0.8743 - val_loss: 0.8685 - val_acc: 0.8906
Epoch 5/16
 - 2s - loss: 6.8406e-06 - acc: 0.9026 - val_loss: 0.7285 - val_acc: 0.9126
Epoch 6/16
 - 2s - loss: 6.3304e-06 - acc: 0.9204 - val_loss: 0.6948 - val_acc: 0.9271
Epoch 7/16
 - 2s - loss: 6.0049e-06 - acc: 0.9326 - val_loss: 0.6364 - val_acc: 0.9374
Epoch 8/16
 - 2s - loss: 5.1107e-06 - acc: 0.9415 - val_loss: 0.3231 - val_acc: 0.9451
Epoch 9/16
 - 2s - loss: 4.4337e-06 - acc: 0.9482 - val_loss: 0.1685 - val_acc: 0.9511
Epoch 10/16
 - 2s - loss: 4.1536e-06 - acc: 0.9536 - val_loss: 0.1034 -